# 🧪 Pruebas Unitarias - Sistema CRUD Oracle-Python

Este notebook contiene todas las pruebas unitarias para el sistema de gestión de datos con Oracle Database.

## 📋 Componentes a Probar:
- **Base de Datos**: Conexión a Oracle Database
- **APIs REST**: 7 endpoints (empleados, fabricas, productos, clientes, sucursales, tarjetas)  
- **Operaciones CRUD**: Create, Read, Update, Delete
- **Secuencias Oracle**: Validación de auto-incremento
- **Manejo de Errores**: Validación de excepciones
- **Integración**: Pruebas end-to-end

## 🎯 Objetivo:
Verificar que todo el sistema funciona correctamente y cumple con los requisitos.

In [ ]:
# 📦 Importar Bibliotecas de Testing
import unittest
import requests
import json
import time
import sys
import os
from unittest.mock import patch, MagicMock
from datetime import datetime

# Agregar el directorio del proyecto al path
sys.path.append(os.getcwd())

# Importar módulos del proyecto
try:
    from database import OracleDatabase
    from models import Empleado, Fabrica, Producto, Cliente, Sucursal, Tarjeta
    import app
    print("✅ Módulos del proyecto importados correctamente")
except ImportError as e:
    print(f"❌ Error al importar módulos: {e}")

# Configuración de la aplicación de pruebas
TEST_BASE_URL = "http://localhost:5000"
print(f"🔧 URL base para pruebas: {TEST_BASE_URL}")

In [ ]:
# 🔧 Setup del Entorno de Pruebas

class TestConfig:
    """Configuración para las pruebas"""
    DB_HOST = "DESKTOP-IFVF9EL"
    DB_PORT = 1521
    DB_SERVICE = "orcl"
    DB_USER = "SYSTEM"
    DB_PASSWORD = "Azucar123"
    
    # Datos de prueba
    TEST_EMPLEADO = {
        "nombre": "Test Employee",
        "sucursal": "Sur",
        "cargo": "Tester"
    }
    
    TEST_FABRICA = {
        "nombre": "Test Factory",
        "pais": "Test Country"
    }
    
    TEST_PRODUCTO = {
        "nombre": "Test Product",
        "precio": 99.99,
        "categoria": "Test Category"
    }

def wait_for_server(url, timeout=30):
    """Espera a que el servidor esté disponible"""
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                return True
        except requests.exceptions.RequestException:
            time.sleep(1)
    return False

def cleanup_test_data():
    """Limpia datos de prueba (opcional)"""
    print("🧹 Preparando entorno de pruebas...")

print("✅ Configuración de pruebas lista")

In [ ]:
# 🏗️ Clase de Pruebas para Base de Datos

class TestDatabaseConnection(unittest.TestCase):
    """Pruebas para la conexión a Oracle Database"""
    
    def setUp(self):
        """Configuración antes de cada prueba"""
        self.db = None
    
    def tearDown(self):
        """Limpieza después de cada prueba"""
        if self.db:
            self.db.close_connection()
    
    def test_database_connection(self):
        """Prueba la conexión a Oracle Database"""
        try:
            self.db = OracleDatabase(
                host=TestConfig.DB_HOST,
                port=TestConfig.DB_PORT,
                service_name=TestConfig.DB_SERVICE,
                user=TestConfig.DB_USER,
                password=TestConfig.DB_PASSWORD
            )
            self.assertIsNotNone(self.db.connection)
            print("✅ Conexión a base de datos exitosa")
        except Exception as e:
            self.fail(f"❌ Fallo en conexión a base de datos: {e}")
    
    def test_database_query(self):
        """Prueba una consulta simple a la base de datos"""
        try:
            self.db = OracleDatabase(
                host=TestConfig.DB_HOST,
                port=TestConfig.DB_PORT,
                service_name=TestConfig.DB_SERVICE,
                user=TestConfig.DB_USER,
                password=TestConfig.DB_PASSWORD
            )
            result = self.db.execute_query("SELECT 1 FROM DUAL")
            self.assertIsNotNone(result)
            self.assertEqual(len(result), 1)
            print("✅ Consulta de prueba exitosa")
        except Exception as e:
            self.fail(f"❌ Fallo en consulta de prueba: {e}")

# Ejecutar pruebas de base de datos
print("🔍 Ejecutando pruebas de conexión a base de datos...")
suite = unittest.TestLoader().loadTestsFromTestCase(TestDatabaseConnection)
runner = unittest.TextTestRunner(verbosity=2)
result = runner.run(suite)
print(f"📊 Resultados: {result.testsRun} pruebas, {len(result.failures)} fallos, {len(result.errors)} errores")

In [ ]:
# 🌐 Clase de Pruebas para APIs REST

class TestAPIEndpoints(unittest.TestCase):
    """Pruebas para todos los endpoints de la API"""
    
    @classmethod
    def setUpClass(cls):
        """Configuración una vez para toda la clase"""
        print("🚀 Verificando que el servidor esté ejecutándose...")
        if not wait_for_server(TEST_BASE_URL):
            raise unittest.SkipTest("Servidor no disponible en localhost:5000")
        print("✅ Servidor disponible")
    
    def test_home_endpoint(self):
        """Prueba el endpoint principal"""
        response = requests.get(f"{TEST_BASE_URL}/")
        self.assertEqual(response.status_code, 200)
        print("✅ Endpoint principal funcionando")
    
    def test_empleados_get(self):
        """Prueba GET /api/empleados"""
        response = requests.get(f"{TEST_BASE_URL}/api/empleados")
        self.assertEqual(response.status_code, 200)
        data = response.json()
        self.assertIn('data', data)
        self.assertIsInstance(data['data'], list)
        print(f"✅ GET empleados: {len(data['data'])} registros")
    
    def test_fabricas_get(self):
        """Prueba GET /api/fabricas"""
        response = requests.get(f"{TEST_BASE_URL}/api/fabricas")
        self.assertEqual(response.status_code, 200)
        data = response.json()
        self.assertIn('data', data)
        self.assertIsInstance(data['data'], list)
        print(f"✅ GET fabricas: {len(data['data'])} registros")
    
    def test_productos_get(self):
        """Prueba GET /api/productos"""
        response = requests.get(f"{TEST_BASE_URL}/api/productos")
        self.assertEqual(response.status_code, 200)
        data = response.json()
        self.assertIn('data', data)
        self.assertIsInstance(data['data'], list)
        print(f"✅ GET productos: {len(data['data'])} registros")
    
    def test_clientes_get(self):
        """Prueba GET /api/clientes"""
        response = requests.get(f"{TEST_BASE_URL}/api/clientes")
        self.assertEqual(response.status_code, 200)
        data = response.json()
        self.assertIn('data', data)
        self.assertIsInstance(data['data'], list)
        print(f"✅ GET clientes: {len(data['data'])} registros")
    
    def test_sucursales_get(self):
        """Prueba GET /api/sucursales"""
        response = requests.get(f"{TEST_BASE_URL}/api/sucursales")
        self.assertEqual(response.status_code, 200)
        data = response.json()
        self.assertIn('data', data)
        self.assertIsInstance(data['data'], list)
        print(f"✅ GET sucursales: {len(data['data'])} registros")
    
    def test_tarjetas_get(self):
        """Prueba GET /api/tarjetas"""
        response = requests.get(f"{TEST_BASE_URL}/api/tarjetas")
        self.assertEqual(response.status_code, 200)
        data = response.json()
        self.assertIn('data', data)
        self.assertIsInstance(data['data'], list)
        print(f"✅ GET tarjetas: {len(data['data'])} registros")

# Ejecutar pruebas de API
print("🔍 Ejecutando pruebas de APIs REST...")
suite = unittest.TestLoader().loadTestsFromTestCase(TestAPIEndpoints)
runner = unittest.TextTestRunner(verbosity=2)
result = runner.run(suite)
print(f"📊 Resultados: {result.testsRun} pruebas, {len(result.failures)} fallos, {len(result.errors)} errores")

In [ ]:
# 🔄 Pruebas CRUD Completas

class TestCRUDOperations(unittest.TestCase):
    """Pruebas para operaciones Create, Read, Update, Delete"""
    
    @classmethod
    def setUpClass(cls):
        """Configuración una vez para toda la clase"""
        if not wait_for_server(TEST_BASE_URL):
            raise unittest.SkipTest("Servidor no disponible")
        cls.created_ids = {}  # Para rastrear IDs creados
    
    def test_empleado_crud_cycle(self):
        """Prueba ciclo completo CRUD para empleados"""
        # CREATE
        empleado_data = TestConfig.TEST_EMPLEADO.copy()
        response = requests.post(f"{TEST_BASE_URL}/api/empleados", 
                               json=empleado_data)
        self.assertEqual(response.status_code, 201)
        created = response.json()
        empleado_id = created.get('id') or created.get('ID_EMPLEADO')
        self.assertIsNotNone(empleado_id)
        print(f"✅ Empleado creado con ID: {empleado_id}")
        
        # READ
        response = requests.get(f"{TEST_BASE_URL}/api/empleados/{empleado_id}")
        self.assertEqual(response.status_code, 200)
        empleado = response.json()
        print("✅ Empleado leído correctamente")
        
        # UPDATE
        empleado_data['cargo'] = 'Senior Tester'
        response = requests.put(f"{TEST_BASE_URL}/api/empleados/{empleado_id}", 
                              json=empleado_data)
        self.assertIn(response.status_code, [200, 204])
        print("✅ Empleado actualizado correctamente")
        
        # DELETE
        response = requests.delete(f"{TEST_BASE_URL}/api/empleados/{empleado_id}")
        self.assertIn(response.status_code, [200, 204])
        print("✅ Empleado eliminado correctamente")
    
    def test_fabrica_crud_cycle(self):
        """Prueba ciclo completo CRUD para fábricas"""
        # CREATE
        fabrica_data = TestConfig.TEST_FABRICA.copy()
        response = requests.post(f"{TEST_BASE_URL}/api/fabricas", 
                               json=fabrica_data)
        self.assertEqual(response.status_code, 201)
        created = response.json()
        fabrica_id = created.get('id') or created.get('ID_FABRICA')
        self.assertIsNotNone(fabrica_id)
        print(f"✅ Fábrica creada con ID: {fabrica_id}")
        
        # READ
        response = requests.get(f"{TEST_BASE_URL}/api/fabricas/{fabrica_id}")
        self.assertEqual(response.status_code, 200)
        print("✅ Fábrica leída correctamente")
        
        # UPDATE
        fabrica_data['pais'] = 'Updated Country'
        response = requests.put(f"{TEST_BASE_URL}/api/fabricas/{fabrica_id}", 
                              json=fabrica_data)
        self.assertIn(response.status_code, [200, 204])
        print("✅ Fábrica actualizada correctamente")
        
        # DELETE
        response = requests.delete(f"{TEST_BASE_URL}/api/fabricas/{fabrica_id}")
        self.assertIn(response.status_code, [200, 204])
        print("✅ Fábrica eliminada correctamente")

# Ejecutar pruebas CRUD
print("🔍 Ejecutando pruebas CRUD...")
suite = unittest.TestLoader().loadTestsFromTestCase(TestCRUDOperations)
runner = unittest.TextTestRunner(verbosity=2)
result = runner.run(suite)
print(f"📊 Resultados: {result.testsRun} pruebas, {len(result.failures)} fallos, {len(result.errors)} errores")

In [ ]:
# ✅ Pruebas de Validación de Datos

class TestDataValidation(unittest.TestCase):
    """Pruebas para validación de datos y manejo de errores"""
    
    @classmethod
    def setUpClass(cls):
        if not wait_for_server(TEST_BASE_URL):
            raise unittest.SkipTest("Servidor no disponible")
    
    def test_invalid_empleado_data(self):
        """Prueba datos inválidos para empleado"""
        # Datos faltantes
        invalid_data = {"nombre": ""}  # nombre vacío
        response = requests.post(f"{TEST_BASE_URL}/api/empleados", 
                               json=invalid_data)
        self.assertNotEqual(response.status_code, 201)
        print("✅ Validación correcta para datos inválidos de empleado")
    
    def test_invalid_producto_price(self):
        """Prueba precio inválido para producto"""
        invalid_data = {
            "nombre": "Test Product",
            "precio": -10,  # precio negativo
            "categoria": "Test"
        }
        response = requests.post(f"{TEST_BASE_URL}/api/productos", 
                               json=invalid_data)
        # Puede ser 400 (Bad Request) o el sistema puede manejar esto
        print(f"✅ Respuesta para precio inválido: {response.status_code}")
    
    def test_nonexistent_resource(self):
        """Prueba acceso a recursos inexistentes"""
        response = requests.get(f"{TEST_BASE_URL}/api/empleados/99999")
        self.assertEqual(response.status_code, 404)
        print("✅ Manejo correcto de recurso inexistente")
    
    def test_invalid_endpoint(self):
        """Prueba endpoint inexistente"""
        response = requests.get(f"{TEST_BASE_URL}/api/invalid_endpoint")
        self.assertEqual(response.status_code, 404)
        print("✅ Manejo correcto de endpoint inexistente")

# 🛡️ Pruebas de Manejo de Errores

class TestErrorHandling(unittest.TestCase):
    """Pruebas para manejo de errores y excepciones"""
    
    @classmethod
    def setUpClass(cls):
        if not wait_for_server(TEST_BASE_URL):
            raise unittest.SkipTest("Servidor no disponible")
    
    def test_malformed_json(self):
        """Prueba JSON malformado"""
        response = requests.post(f"{TEST_BASE_URL}/api/empleados", 
                               data="invalid json",
                               headers={'Content-Type': 'application/json'})
        self.assertEqual(response.status_code, 400)
        print("✅ Manejo correcto de JSON malformado")
    
    def test_wrong_http_method(self):
        """Prueba método HTTP incorrecto"""
        response = requests.patch(f"{TEST_BASE_URL}/api/empleados")
        self.assertIn(response.status_code, [405, 404])  # Method Not Allowed
        print("✅ Manejo correcto de método HTTP incorrecto")

# Ejecutar pruebas de validación
print("🔍 Ejecutando pruebas de validación de datos...")
suite1 = unittest.TestLoader().loadTestsFromTestCase(TestDataValidation)
runner = unittest.TextTestRunner(verbosity=2)
result1 = runner.run(suite1)

print("🔍 Ejecutando pruebas de manejo de errores...")
suite2 = unittest.TestLoader().loadTestsFromTestCase(TestErrorHandling)
result2 = runner.run(suite2)

total_tests = result1.testsRun + result2.testsRun
total_failures = len(result1.failures) + len(result2.failures)
total_errors = len(result1.errors) + len(result2.errors)

print(f"📊 Resultados totales: {total_tests} pruebas, {total_failures} fallos, {total_errors} errores")

In [ ]:
# 🏃‍♂️ Ejecutar Suite Completa de Pruebas

def run_complete_test_suite():
    """Ejecuta todas las pruebas del proyecto"""
    print("🚀 INICIANDO SUITE COMPLETA DE PRUEBAS")
    print("=" * 60)
    
    # Lista de todas las clases de prueba
    test_classes = [
        TestDatabaseConnection,
        TestAPIEndpoints, 
        TestCRUDOperations,
        TestDataValidation,
        TestErrorHandling
    ]
    
    # Resultados consolidados
    total_tests = 0
    total_failures = 0
    total_errors = 0
    
    # Ejecutar cada clase de prueba
    for test_class in test_classes:
        print(f"\n📋 Ejecutando: {test_class.__name__}")
        print("-" * 40)
        
        suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
        runner = unittest.TextTestRunner(verbosity=1, stream=open(os.devnull, 'w'))
        result = runner.run(suite)
        
        # Acumular resultados
        total_tests += result.testsRun
        total_failures += len(result.failures)
        total_errors += len(result.errors)
        
        # Mostrar resumen de esta clase
        status = "✅" if len(result.failures) == 0 and len(result.errors) == 0 else "❌"
        print(f"{status} {test_class.__name__}: {result.testsRun} pruebas, {len(result.failures)} fallos, {len(result.errors)} errores")
        
        # Mostrar detalles de fallos si los hay
        if result.failures:
            print("   ⚠️  Fallos:")
            for test, traceback in result.failures:
                print(f"      - {test}")
        
        if result.errors:
            print("   🚨 Errores:")
            for test, traceback in result.errors:
                print(f"      - {test}")
    
    return total_tests, total_failures, total_errors

# Ejecutar suite completa
tests, failures, errors = run_complete_test_suite()

print("\n" + "=" * 60)
print("📊 RESUMEN FINAL DE PRUEBAS")
print("=" * 60)
print(f"📈 Total de pruebas ejecutadas: {tests}")
print(f"✅ Pruebas exitosas: {tests - failures - errors}")
print(f"❌ Pruebas fallidas: {failures}")
print(f"🚨 Errores: {errors}")

# Calcular porcentaje de éxito
if tests > 0:
    success_rate = ((tests - failures - errors) / tests) * 100
    print(f"🎯 Tasa de éxito: {success_rate:.1f}%")
    
    if success_rate >= 90:
        print("🏆 ¡EXCELENTE! El sistema está muy bien probado")
    elif success_rate >= 70:
        print("👍 BUENO. Algunas áreas necesitan atención")
    else:
        print("⚠️  NECESITA MEJORAS. Revisar fallos y errores")
else:
    print("⚠️  No se ejecutaron pruebas")

print("=" * 60)

In [ ]:
# 📊 Generar Reporte Detallado de Pruebas

def generate_detailed_report():
    """Genera un reporte detallado con métricas y estadísticas"""
    print("📋 REPORTE DETALLADO DEL SISTEMA")
    print("=" * 60)
    
    # Verificar disponibilidad del sistema
    print("🔍 VERIFICACIÓN DEL SISTEMA:")
    try:
        response = requests.get(f"{TEST_BASE_URL}/", timeout=5)
        if response.status_code == 200:
            print("   ✅ Servidor Flask disponible")
        else:
            print(f"   ⚠️  Servidor responde con código {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"   ❌ Servidor no disponible: {e}")
    
    # Verificar cada endpoint
    print("\n🌐 ESTADO DE ENDPOINTS:")
    endpoints = [
        "/api/empleados",
        "/api/fabricas", 
        "/api/productos",
        "/api/clientes",
        "/api/sucursales",
        "/api/tarjetas"
    ]
    
    endpoint_status = {}
    for endpoint in endpoints:
        try:
            start_time = time.time()
            response = requests.get(f"{TEST_BASE_URL}{endpoint}", timeout=10)
            response_time = (time.time() - start_time) * 1000  # en ms
            
            if response.status_code == 200:
                data = response.json()
                count = len(data.get('data', []))
                endpoint_status[endpoint] = {
                    'status': '✅',
                    'records': count,
                    'response_time': response_time
                }
                print(f"   ✅ {endpoint}: {count} registros ({response_time:.0f}ms)")
            else:
                endpoint_status[endpoint] = {
                    'status': '❌',
                    'records': 0,
                    'response_time': response_time
                }
                print(f"   ❌ {endpoint}: Error {response.status_code}")
        except Exception as e:
            endpoint_status[endpoint] = {
                'status': '🚨',
                'records': 0,
                'response_time': 0
            }
            print(f"   🚨 {endpoint}: Error - {e}")
    
    # Estadísticas de performance
    print("\\n⚡ MÉTRICAS DE PERFORMANCE:")
    response_times = [ep['response_time'] for ep in endpoint_status.values() if ep['response_time'] > 0]
    if response_times:
        avg_response = sum(response_times) / len(response_times)
        max_response = max(response_times)
        min_response = min(response_times)
        print(f"   📊 Tiempo promedio de respuesta: {avg_response:.0f}ms")
        print(f"   📊 Tiempo máximo de respuesta: {max_response:.0f}ms")
        print(f"   📊 Tiempo mínimo de respuesta: {min_response:.0f}ms")
        
        if avg_response < 500:
            print("   🚀 Performance: EXCELENTE")
        elif avg_response < 1000:
            print("   👍 Performance: BUENA")
        else:
            print("   ⚠️  Performance: NECESITA OPTIMIZACIÓN")
    
    # Resumen de datos
    print("\\n📈 RESUMEN DE DATOS:")
    total_records = sum(ep['records'] for ep in endpoint_status.values())
    working_endpoints = sum(1 for ep in endpoint_status.values() if ep['status'] == '✅')
    
    print(f"   📊 Total de registros en sistema: {total_records}")
    print(f"   📊 Endpoints funcionando: {working_endpoints}/{len(endpoints)}")
    print(f"   📊 Disponibilidad del sistema: {(working_endpoints/len(endpoints)*100):.1f}%")
    
    # Recomendaciones
    print("\\n💡 RECOMENDACIONES:")
    if working_endpoints == len(endpoints):
        print("   ✨ Sistema funcionando óptimamente")
    else:
        print("   🔧 Revisar endpoints con fallos")
    
    if total_records == 0:
        print("   📝 Considerar agregar datos de prueba")
    
    print("=" * 60)
    
    return endpoint_status

# Generar reporte
report_data = generate_detailed_report()

# Timestamp del reporte
print(f"\\n🕐 Reporte generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🏁 PRUEBAS UNITARIAS COMPLETADAS")

## 📚 Instrucciones de Uso

### 🚀 Cómo ejecutar las pruebas:

1. **Antes de ejecutar las pruebas:**
   ```bash
   # Asegúrate de que el servidor Flask esté ejecutándose
   python app.py
   ```

2. **Ejecutar pruebas individuales:**
   - Ejecuta cada celda de Python individualmente
   - Observa los resultados de cada conjunto de pruebas

3. **Ejecutar suite completa:**
   - Ejecuta la celda "Ejecutar Suite Completa de Pruebas"
   - Revisa el reporte final

### 🔧 Configuración:
- **Base de datos**: Oracle Database en `DESKTOP-IFVF9EL:1521/orcl`
- **Servidor**: Flask en `http://localhost:5000`
- **Usuario**: SYSTEM

### 📊 Tipos de pruebas incluidas:

| Categoría | Descripción | Objetivo |
|-----------|-------------|----------|
| **Database** | Conexión y consultas a Oracle | Verificar conectividad |
| **API REST** | Endpoints GET para todas las tablas | Verificar disponibilidad |
| **CRUD** | Operaciones completas Create/Read/Update/Delete | Verificar funcionalidad |
| **Validación** | Datos inválidos y casos edge | Verificar robustez |
| **Errores** | Manejo de excepciones | Verificar estabilidad |

### 🎯 Métricas evaluadas:
- ✅ **Funcionalidad**: Todas las operaciones funcionan
- ⚡ **Performance**: Tiempos de respuesta < 500ms
- 🛡️ **Robustez**: Manejo correcto de errores
- 📊 **Cobertura**: Todas las tablas y endpoints

### 🚨 Solución de problemas:
- **Servidor no disponible**: Verificar que `python app.py` esté ejecutándose
- **Error de base de datos**: Verificar credenciales y conexión Oracle
- **Fallos en CRUD**: Verificar que las secuencias Oracle existan
- **Timeouts**: Aumentar timeout o verificar performance de BD